### Download SECC Data from Dataverse

In [ ]:
# Dataverse API Token
# https://dataverse.harvard.edu/dataverseuser.xhtml?selectTab=apiTokenTab
token = '<API TOKEN>'

In [ ]:
from pyDataverse.api import Api
from pyDataverse.models import Dataverse

In [ ]:
api = Api('https://dataverse.harvard.edu/', token)
api.status

In [ ]:
resp = api.get_dataset('doi:10.7910/DVN/LIIBNB')
j = resp.json()
j

In [ ]:
files = []
for file in j['data']['latestVersion']['files']:
    fid = file['dataFile']['id']
    fn = file['dataFile']['filename']
    print(fid, fn)
    files.append((fid, fn))

In [ ]:
import requests
def download_file(url, fn):
    local_filename = fn # url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return local_filename

In [ ]:
for fid, fn in files:
    print(fn)
    if not fn.startswith('secc_deduped.csv.gz'):
        continue
    url = 'https://dataverse.harvard.edu//api/v1/access/datafile/%s?key=%s' % (fid, token)
    download_file(url, fn)
    #break

### Join all parts

In [ ]:
!cat secc_deduped.csv.gz.part* > /opt/data/secc/v2/secc_deduped.csv.gz

In [ ]:
!rm -rf secc_deduped.csv.gz.part*